# Installing dependencies and HFLLM Setup

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install fastapi uvicorn pyngrok langchain langchain-community langchain-core transformers==4.52.4 torch streamlit pyyaml accelerate peft bitsandbytes

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 71.2 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 73.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.8/450.8 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 22

In [3]:
!pip install -U bitsandbytes

# Model 1 : (CodeLlama-7b-Instruct-hf) for Code analysis and architecture

In [4]:
from langchain.llms.base import LLM
from typing import Any

class CustomHFLLM(LLM):
    def _call(self, prompt: str, stop: Any = None) -> str:
        return generate_text(prompt, max_length=500)
    
    @property
    def _llm_type(self) -> str:
        return "custom_huggingface"

llm = CustomHFLLM()


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

# Load model and tokenizer
model_name = "codellama/CodeLlama-7b-Instruct-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Quantization
bnb_cfg = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_cfg,
    device_map="auto"
).eval()
def generate_text(prompt, max_length=300):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            pad_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/646 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

# Model 2 : (Mistral-Nemo-Instruct-2407) for Documentation 

In [6]:
class CustomHFLLM_docs(LLM):
    def _call(self, prompt: str, stop: Any = None) -> str:
        return generate_text_docs(prompt, max_length=500)
    
    @property
    def _llm_type(self) -> str:
        return "custom_huggingface"
llm_docs = CustomHFLLM_docs()

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

# Load model and tokenizer
model_name = "mistralai/Mistral-Nemo-Instruct-2407"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Quantization
bnb_cfg = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_cfg,
    device_map="auto"
).eval()
def generate_text_docs(prompt, max_length=300):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            pad_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

# RESPONSE SCHEMAS & OUTPUT PARSERS

In [14]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

# ANALYSIS PARSER
analysis_schemas = [
    ResponseSchema(name="purpose", description="Main purpose and functionality"),
    ResponseSchema(name="key_components", description="Key components or functions"),
    ResponseSchema(name="dependencies", description="Dependencies needed"),
    ResponseSchema(name="config", description="Configuration requirements"),
]

analysis_parser = StructuredOutputParser.from_response_schemas(analysis_schemas)
analysis_format = analysis_parser.get_format_instructions()

# ARCHITECTURE PARSER
architecture_schemas = [
    ResponseSchema(name="pattern", description="Architecture pattern"),
    ResponseSchema(name="components", description="Main components and relationships"),
    ResponseSchema(name="data_flow", description="Data flow"),
    ResponseSchema(name="design", description="Key design decisions"),
]

architecture_parser = StructuredOutputParser.from_response_schemas(architecture_schemas)
architecture_format = architecture_parser.get_format_instructions()

# DOCUMENTATION PARSER
docs_schemas = [
    ResponseSchema(name="markdown", description="The final generated markdown documentation")
]

docs_parser = StructuredOutputParser.from_response_schemas(docs_schemas)
docs_format = docs_parser.get_format_instructions()


# LLM Chains 

In [15]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Chain 1: Code Analysis Chain - FIXED
analysis_prompt = PromptTemplate(
    input_variables=["code","format_instructions"],
    partial_variables={"format_instructions": analysis_format},

    template="""Analyze this code and extract key information:

CODE:
{code}

Extract ONLY these 4 things:
1. Main purpose and functionality
2. Key components/functions  
3. Dependencies needed
4. Configuration requirements

Respond ONLY in JSON format as follows:

{format_instructions}
"""
)

# Chain 2: Architecture Detection Chain - FIXED  
architecture_prompt = PromptTemplate(
    input_variables=["code"],
    partial_variables={"format_instructions": architecture_format},
    template="""Analyze the architecture pattern in this code:

{code}

Identify these 4 things:
1. Architecture pattern (MVC, Microservices, etc.)
2. Main components and their relationships
3. Data flow
4. Key design decisions

Respond ONLY in JSON format as follows:

{format_instructions}
"""
)

# Chain 3: Documentation Generation Chain - FIXED
docs_prompt = PromptTemplate( input_variables=["analysis", "architecture", "code"],
                             partial_variables={"format_instructions": docs_format},
                             template = """
Generate comprehensive markdown documentation using this analysis:

CODE ANALYSIS: {analysis}
ARCHITECTURE: {architecture}
ORIGINAL CODE: {code}

Create well-structured markdown with these sections:

# Overview
# Architecture
# Setup Instructions
# Usage Examples
# API Documentation (if applicable)

IMPORTANT:
- Use proper markdown formatting with headers, code blocks, and lists
- Do NOT generate excessive backticks or empty code blocks
- Write clean, professional documentation
- Include actual code examples where appropriate

Respond ONLY in JSON format as follows:
{format_instructions}
"""

                            )


# 3) CHAINS


analysis_chain = LLMChain(llm=llm, prompt=analysis_prompt)
architecture_chain = LLMChain(llm=llm, prompt=architecture_prompt)
docs_chain = LLMChain(llm=llm_docs, prompt=docs_prompt)



/tmp/ipykernel_48/1637582774.py:80: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  analysis_chain = LLMChain(llm=llm, prompt=analysis_prompt)


# JSON Extraction

In [20]:
import regex as re
def extract_json_block(text,output_parser):
    pattern = r'```json\s*(.*?)\s*```'
    matches = re.findall(pattern, text, re.DOTALL)
    json_text = f"```json\n{matches[-1]}\n```"
    output_data = output_parser.parse(json_text)
    return output_data

# Full Pipeline

In [21]:
def run_all(code: str):
    print(f"code: {code}")

    raw_analysis = analysis_chain.run(code)
    analysis = extract_json_block(raw_analysis,analysis_parser)
    
    raw_architecture = architecture_chain.run(code)
    architecture = extract_json_block(raw_architecture,architecture_parser)
    
    raw_docs = docs_chain.run({
            "code": code,
            "analysis": analysis,
            "architecture": architecture
        })
    pattern = r'(\{\s*"markdown"\s*:\s*"(?:[^"\\]|\\.|\\\n)*"\s*\})'
    matches = re.findall(pattern,raw_docs , re.DOTALL)
    json_text = f"```json\n{matches[-1]}\n```"
    doc = docs_parser.parse(json_text)
    return analysis , architecture , doc 


## Usage example

In [22]:
code = """#include <iostream>
#include <vector>
#include <string>

class User {
private:
    int id;
    std::string name;

public:
    User(int uid, const std::string &uname) : id(uid), name(uname) {}

    void print() const {
        std::cout << "User ID: " << id << ", Name: " << name << std::endl;
    }
};

class UserManager {
private:
    std::vector<User> users;

public:
    void addUser(int id, const std::string &name) {
        users.emplace_back(id, name);
    }

    void listUsers() const {
        if (users.empty()) {
            std::cout << "No users available.\n";
            return;
        }
        std::cout << "=== User List ===\n";
        for (const auto &user : users) {
            user.print();
        }
    }
};

int main() {
    UserManager manager;
    bool running = true;

    while (running) {
        std::cout << "\n1. Add User\n2. List Users\n3. Exit\nChoose option: ";
        int choice;
        std::cin >> choice;

        if (choice == 1) {
            int id;
            std::string name;
            std::cout << "Enter user ID: ";
            std::cin >> id;
            std::cout << "Enter name: ";
            std::cin >> name;
            manager.addUser(id, name);

        } else if (choice == 2) {
            manager.listUsers();

        } else if (choice == 3) {
            running = false;
            std::cout << "Exiting...\n";

        } else {
            std::cout << "Invalid option.\n";
        }
    }

    return 0;
}
"""

In [23]:
analysis , architecture , doc = run_all(code)

code: #include <iostream>
#include <vector>
#include <string>

class User {
private:
    int id;
    std::string name;

public:
    User(int uid, const std::string &uname) : id(uid), name(uname) {}

    void print() const {
        std::cout << "User ID: " << id << ", Name: " << name << std::endl;
    }
};

class UserManager {
private:
    std::vector<User> users;

public:
    void addUser(int id, const std::string &name) {
        users.emplace_back(id, name);
    }

    void listUsers() const {
        if (users.empty()) {
            std::cout << "No users available.
";
            return;
        }
        std::cout << "=== User List ===
";
        for (const auto &user : users) {
            user.print();
        }
    }
};

int main() {
    UserManager manager;
    bool running = true;

    while (running) {
        std::cout << "
1. Add User
2. List Users
3. Exit
Choose option: ";
        int choice;
        std::cin >> choice;

        if (choice == 1) {
            int id;
   

In [24]:
print(analysis)

{'purpose': 'This code implements a simple User Management System where users can be added and listed.', 'key_components': 'User class with id and name attributes, UserManager class with addUser() and listUsers() functions.', 'dependencies': 'Standard Library includes <iostream>, <vector>, <string>.', 'config': 'No configuration requirements.'}


In [25]:
print(architecture)

{'pattern': 'Singleton', 'components': 'UserManager is a Singleton managing User objects.', 'data_flow': 'User data flows from UserManager to User objects.', 'design': 'UserManager uses std::vector to store User objects.'}


In [26]:
print(doc)

{'markdown': '# Overview\n\nThis code implements a simple User Management System where users can be added and listed.\n\n## Architecture\n\n### Pattern\n\nThe Singleton pattern is used in this codebase.\n\n### Components\n\n- `UserManager`: A Singleton managing `User` objects.\n\n### Data Flow\n\nUser data flows from `UserManager` to `User` objects.\n\n### Design\n\n`UserManager` uses `std::vector` to store `User` objects.\n\n## Setup Instructions\n\n1. Include the necessary headers:\n\n```cpp\n#include <iostream>\n#include <vector>\n#include <string>\n```\n\n2. Define the `User` class with `id` and `name` attributes.\n\n3. Define the `UserManager` class with `addUser()` and `listUsers()` functions.\n\n## Usage Examples\n\nHere are some usage examples:\n\n- Add a user:\n\n```cpp\nmanager.addUser(1, "John Doe");\n```\n\n- List users:\n\n```cpp\nmanager.listUsers();\n```\n\n## API Documentation\n\n### User\n\n- `User(int uid, const std::string &uname)`: Constructor to initialize `id` and

# 

In [27]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
API_KEY = user_secrets.get_secret("API_KEY")
NGROK_TOKEN = user_secrets.get_secret("NGROK_TOKEN")


In [28]:
from fastapi import FastAPI, Request, HTTPException
import uvicorn, threading, time, socket
from pyngrok import ngrok, conf

app = FastAPI()

@app.post("/generate")
async def gen(req: Request):
    if req.headers.get("authorization") != f"Bearer {API_KEY}":
        raise HTTPException(status_code=401, detail="Unauthorized")
    data = await req.json()
    return {
        "response": run_all(
            data.get("prompt", ""),
        )
    }

In [29]:
def free_port():
    s = socket.socket()
    s.bind(('', 0))
    port = s.getsockname()[1]
    s.close()
    return port

port = free_port()
conf.get_default().auth_token = NGROK_TOKEN
public_url = ngrok.connect(port).public_url
print("Your public URL:", public_url)

def run(): uvicorn.run(app, host="0.0.0.0", port=port)
threading.Thread(target=run, daemon=True).start()
time.sleep(1)

Your public URL: https://3a9d52169f9e.ngrok-free.app                                                


INFO:     Started server process [48]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:45053 (Press CTRL+C to quit)


code: from flask import Flask, jsonify

app = Flask(__name__)

@app.route('/api/users')
def get_users():
    return jsonify([{'id': 1, 'name': 'John'}])

if __name__ == '__main__':
    app.run(debug=True)
INFO:     102.45.25.101:0 - "POST /generate HTTP/1.1" 200 OK
code: import streamlit as st
import requests
import os
from dotenv import load_dotenv
import tempfile

# --- Configuration ---
load_dotenv()
API_KEY = os.getenv("API_KEY")
BACKEND_URL = os.getenv("BACKEND_URL")

# --- Page Setup ---
st.set_page_config(
    page_title="AI Documentation Assistant",
    page_icon="🤖",
    layout="wide"
)

# --- Sidebar ---
st.sidebar.title("About AI Documentation Assistant")
st.sidebar.info(
    """
    This assistant helps you automatically generate **code analysis**, **architecture overview**, and **documentation** from your code or prompt.

    **Tabs:**
    - 📝 **Analysis**: Purpose, key components, dependencies, and configuration.
    - 🏗️ **Architecture**: Design patterns, components, dat